In [1]:
import pandas as pd
import seaborn as sns


In [3]:
# 1 데이터 준비

# seaborn에 있는 타이타닉 데이터 데이터프레임 변환
df = sns.load_dataset('titanic')

In [4]:
# 출력할 열의 개수 한도 늘리기
pd.set_option('display.max_columns', 15)

In [6]:
# 2. 데이터 탐색 / 전처리

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [9]:
# nan값이 많은 deck 삭제, embarked와 내용이 겹치는 embark_town 열 살제

rdf = df.drop(['deck', 'embark_town'], axis=1)

In [10]:
# age 열에 나이 데이터가 없는 모든 행 삭제 - age 열(891개 중 177개의 NAN값)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)

In [11]:
# embarked 열의 nan값을 승선 도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq, inplace=True)


In [12]:
#3. 분석에 사용할 열 선택

ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]

In [13]:
# sex, embarked범주형 데이터 숫자형으로 변환하는 더미 변수 만들기(원 핫 인코딩)
onehot_sex = pd.get_dummies(ndf['sex'])#sex열 더미변수화
ndf = pd.concat([ndf, onehot_sex], axis=1)#기존 데이프레임 연결


In [14]:
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')#열 이름에 접두어 town 붙이고 더미변수화
ndf = pd.concat([ndf, onehot_embarked], axis=1)#기존 데이터프레임 연결

In [15]:
#기존 열 삭제
ndf.drop(['sex','embarked'], axis=1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [16]:
# 4. 데이터셋 구분 - 훈련용/검증용

#속성 변수 선택

X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']]#설명변수 X
y = ndf['survived']#예측변수 Y

In [17]:
# 설명변수 데이터를 정규화 (normalization)


from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [18]:
# train data와 test data로 구분(7:3 비율)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

print('train data 개수:', X_train.shape)
print('test data 개수:', X_test.shape)

train data 개수: (499, 9)
test data 개수: (215, 9)


In [ ]:
# 5. 모형학습 및 검증

